# Envio de mensagens via WhatsApp Web

Passos:
1) Criar a janela
2) Botão para selecionar arquivo de clientes
3) Label para mostrar caminho do arquivo de clientes
4) Label para mostrar quantidade total de mensagens a serem enviadas (contar quantos telefones E mensagens não vazias existem)
5) Botão para selecionar imagens
6) Label para mostrar caminho das imagens (apenas um unico string com todas as imagens da lista concatenadas)
7) Botão para iniciar processo de envio
8) Processo:  
    8.1) Apenas imprimir as mensagens  
    8.2) Label com dados atualizados na janela:  
        numero de enviados / numero total  
        percentual enviado 

Bibliotecas

In [1]:
# import libraries

# Basic Tkinter
import tkinter as tk
from tkinter import filedialog as fd
import tkinter.scrolledtext as st

# PIL to show images on Tkinter
from PIL import Image, ImageTk

# Pandas
import pandas as pd

# Just to get image file name from full path
from pathlib import Path

# Time to allow program wait few seconds during Chrome operations
import time

# To allow randomic waiting times (important to avoid Whatsapp account blocking)
import random

# Datetime to store current date of messages sent
from datetime import date

# necessary libraries for Chrome operations:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys


# pip install webdriver_manager
# This librari updates automatically the Browser Manger (in this case, Chrome)
from webdriver_manager.chrome import ChromeDriverManager

# Necessary to convert messages from ASCII text into URL aceptable addresses (convert special characters, spaces, etc)
import urllib


In [2]:
# global variables

# list of images to send
imgs_path = []

# current image on viewer
i = 0


Funções

In [3]:
def show_imgs():
    
    # Event Function to show the i image from list
    
    global tkphoto 
    # Canvas Size
    can_h = 400
    can_w = 400
    
    # esta demorando muito aqui, a solução de usar uma variavel global nao e interessante
    # https://blog.furas.pl/python-tkinter-why-label-doesnt-display-image-bug-with-garbage-collector-in-photoimage-GB.html
    # https://www.c-sharpcorner.com/blogs/basics-for-displaying-image-in-tkinter-python
    
    # Get the element i from list
    photo = Image.open(imgs_path[i])
    
    # Get the picture size (widht, height)
    pic_w, pic_h = photo.size

    # Calculate aspect image ratio
    aspect = pic_w/pic_h

    # if picture is wider than taller, the resizing limit will be the picture widht, limited to canvas width
    if aspect > 1:
        res_w = can_w
        res_h = can_w / aspect
    
    # else, the resizing limit will be the picture height, limited to canvas height
    else:
        res_h = can_h
        res_w = can_h * aspect
    

    # resize picture
    photo = photo.resize((int(res_w),int(res_h)))

    # create the Tkinter picture image object
    tkphoto = ImageTk.PhotoImage(photo) 

    # put the picture into the label
    lbl_photo = tk.Label(image=tkphoto,width=can_w,height=can_h,borderwidth=2,relief='solid')
    lbl_photo.grid(row=1,column=4,rowspan=4, padx=10,pady=10)
    
    # mostrar numero da imagem e o nome
    show_name()
    
    return()

In [4]:
def show_name():
    # Function to show current image number, total number of selected images and current image name (without path)
    # Path.name method extracts the name from a full 
    img_name = '{} de {}: {}'.format(i+1,len(imgs_path),Path(imgs_path[i]).name)
    lbl_imgname = tk.Label(text=img_name,font=('Consolas 10'))
    lbl_imgname.grid(row=5, column=3,columnspan=3,sticky='NSEW',padx=10,pady=10)
    return()
    

In [5]:
def go_rgt():
    # Event function to select next (right) image
    global i
    if i < len(imgs_path)-1:
        i = i + 1
    else:
        i = 0
    
    # Call function to show image number i
    show_imgs()

    return()

In [6]:
def go_lft():
    # Event function to select previous (left) image
    global i
    if i > 0:
        i = i - 1
    else:
        i = len(imgs_path)-1
    
    # Call function to show image number i
    show_imgs()
    
    return()

In [7]:
def sel_imgs():
    # Event function to select image files
        
    # imgs_path is a global list
    global imgs_path

    # i is the number of the image to show
    global i

    images_types = [
            ('Arquivos de imagen','.jpg'),
            ('Arquivos de imagen','.jpeg'),
            ('Arquivos de imagen','.png'),
            ('Arquivos de imagen','.gif'),
            ]

    # Select images
    imgs_path = sorted(list(fd.askopenfilenames(title='Selecione as imagens a enviar',filetypes=images_types)))
    
    # Define show previous image button
    btn_lft = tk.Button(text='<',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_lft)
    btn_lft.grid(row=1,column=3,rowspan=4,sticky='NSEW',padx=10,pady=10)

    # Define show next image button
    btn_rgt = tk.Button(text='>',font=('Consolas 20 bold'),wraplength=100,borderwidth=1,command=go_rgt)
    btn_rgt.grid(row=1,column=5,rowspan=4,sticky='NSEW',padx=10,pady=10)
        
    # Call function to show first image of selected list
    i = 0
    show_imgs()

    return()

In [8]:
def sel_file():
    
    # Event function to select Excel file
    
    # contacts_df is the global dataframe with destinataries names, numbers and messages
    global contacts_df

    # file_path is where file is stored
    global file_path

    file_path = fd.askopenfilename(
        title='Selecione o arquivo Excel com a lista de destinatarios',
        filetypes=[('Arquivo Excel','.xls'),('Arquivo Excel','.xlsx')]
        )
    
    # Read Excel file
    contacts_df = pd.read_excel(file_path, sheet_name='CLIENTES')

    # Remove rows with empty messages
    contacts_df = contacts_df[~contacts_df['MENSAGEM'].isnull()]

    # Reset index
    contacts_df.reset_index(inplace=True)

    # Remove new index column created
    contacts_df.drop(labels='index',axis=1,inplace=True)

    # update informations about number of messages to be sent and
    # inform to click button to start process
    lbl_slctdfile['text'] = 'Serão enviadas {} mensagens.\n Clique em "Enviar Mensagens" para iniciar'.format(contacts_df['MENSAGEM'].count())
    
    return()

Processo Principal

In [9]:
# Function to wait for WhatsApp contacts side bar for x seconds
def wait_wpp_contacts(timetowaith):
    while len(msg_browser.find_elements(By.ID,"side")) < 1:
        time.sleep(timetowaith)
    return()
    

In [10]:
def send_messages():
    
    global msg_browser
    
    # Count total number of messages to send
    msg_total = contacts_df['MENSAGEM'].count()

    # Create intance of Google Chorme browsed
    msg_browser = webdriver.Chrome(ChromeDriverManager().install())
    
    # Navigate to WhatsApp Web
    msg_browser.get("https://web.whatsapp.com/")
    time.sleep(5)

    # Link will open the QR Code authorization
    # Wait until user authorization with cell phone
    
    # Wait to load WhatsApp contacts side bar
    # this indicates it is possible to send messages
    wait_wpp_contacts(2)

    for j, message in enumerate(contacts_df['MENSAGEM']):

        # this version considers all messages are not null
        # dataframe already cleaned up on opening file function

        # Get customer name and number
        name = contacts_df.loc[j,"CLIENTE"]
        phone = contacts_df.loc[j, "TELEFONE"]
        
        # Update status label
        lbl_enviando['text'] = 'Enviando mensagem {} de {}\nPara {} no telefone {}'.format(j+1,msg_total,name,phone)
        mainwindow.update()

        # Convert message from ASCII into URL plain text
        url_message = urllib.parse.quote(f"{message}")

        # construir o link
        link = f"https://web.whatsapp.com/send?phone={phone}&text={url_message}"

        # Try to open link; if phone is wrong, it will generate and error, and pass to next
        try:
            # Get link
            msg_browser.get(link)

            # Wait to load WhatsApp contacts side bar
            # this indicates it is possible to send messages
            wait_wpp_contacts(1)
                
            # Send ENTER
            msg_browser.find_element(By.XPATH,'//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p/span').send_keys(Keys.ENTER)
            
            # Wait to load WhatsApp contacts side bar
            # this indicates it is possible to send messages
            wait_wpp_contacts(1)
            
            # For each image in images list
            for img_file in imgs_path:
        
                # Find attachment button (paperclip) and click on it
                msg_browser.find_element(By.CSS_SELECTOR,"span[data-icon='clip']").click()
                time.sleep(2)

                # Select to find element and send all keys the img_file as keystrokes
                msg_browser.find_element(By.CSS_SELECTOR,"input[type='file']").send_keys(img_file)
                time.sleep(5)

                # Click on send attachment (littel triangle)
                msg_browser.find_element(By.CSS_SELECTOR,"span[data-icon='send']").click()
                time.sleep(1)
            
            # Write on scrolling text box the result of current message sending process
            txt_result = 'recebeu a mensagem'
                            
        except:
            # Write on scrolling text box the result of current message sending process
            txt_result = 'NÃO recebeu a mensagem'

        # Print on terminal
        print('{}:{}: {}'.format(j+1,name,txt_result))
            
        # add to result to data frame
        contacts_df.loc[j,'RESULTADO'] = txt_result

        # Write on scrolling text box the result of current message sending process
        txt_enviados.insert(tk.INSERT,'{}: {} {}\n'.format(j+1,name,txt_result))
        
        # Update Tkinter process
        # note: time.sleep needs to update Tkinter frequently            
        mainwindow.update()

        # Wait a random time before send next.
        # this is important to avoid WhatsApp to cancel the account due to automation
        time.sleep(random.randint(5,10))
    
    # Sending Loop ends here

    lbl_slctdfile['text'] = 'Processo Finalizado'

    # save results dataframe
    result_file = '{}\Resultado Envios {}.xlsx'.format(Path(file_path).parent,date.today())
    contacts_df.to_excel(result_file,index=False)

    # aqui colocar os resultados de envio.
    # investigar como fazer somente leitura.
    # as configrações de state normal e disabled funcionam mas nao permitem alterar pelo programa        
    # txt_enviados.configure(state ='normal')
    # txt_enviados.insert(tk.INSERT,'{} recebeu a mensagem\n'.format(contacts_df['CLIENTE'][j]))
    # txt_enviados.configure(state ='disabled')
    
    return()

Design da janela principal

In [11]:
# criar a janela da aplicação
mainwindow = tk.Tk()

In [12]:
# configurações da janela
# mainwindow.geometry('750x500')
# mainwindow.rowconfigure(0,weight=1)
# mainwindow.columnconfigure([0,1],weight=1)

In [13]:
# Titulo da janela
mainwindow.title("Envio de Mensagens pelo WhatsApp")

''

In [14]:
# Label com titulo principal da janela
lbl_title = tk.Label(text="Envio de Mensagens pelo WhatsApp",font=('Consolas 15 bold underline'),borderwidth=1, relief='solid')
lbl_title.grid(row=0, column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [15]:
# Label de explicação
lbl_desc = tk.Label(text=
    """Este programa envia mensagens pelo WhatsApp Web,
    a partir de uma lista de destinatarios em formato Excel.
    A lista deve conter os campos
    Nome, Telefone e Mensagens, numa folha Clientes"""
    ,font=('Consolas 10'),wraplength=300,borderwidth=1, relief='solid')
lbl_desc.grid(row=1, column=0, columnspan=3,sticky='NSEW',padx=10,pady=10) 

In [16]:
# Label para seleção de arquivo Excel
lbl_file = tk.Label(text='Seleccione o arquivo Excel com os dados:',font=('Consolas 12'),anchor='e')
lbl_file.grid(row=3,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [17]:
# Botao para selecionar o arquivo em Excel
btn_file = tk.Button(text='Clique aqui para seleccionar o arquivo',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_file)
btn_file.grid(row=3,column=2,sticky='NSEW',padx=10,pady=10)

In [18]:
# Caminho e nome do arquivo seleccionado, posteriormente quantidade de mensagens
lbl_slctdfile = tk.Label(text='Nenhum Arquivo Seleccionado',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_slctdfile.grid(row=4,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [19]:
# Label para seleção dos arquivos de imagens
lbl_imgs = tk.Label(text='Seleccione as imagens a enviar:',font=('Consolas 12'),anchor='e')
lbl_imgs.grid(row=2,column=0,columnspan=2,sticky='NSEW',padx=10,pady=10)

In [20]:
# Botão para selecionar imagens
btn_imgs = tk.Button(text='Clique aqui para seleccionar as imagens',font=('Consolas 10 bold'),wraplength=100,borderwidth=1,command=sel_imgs)
btn_imgs.grid(row=2,column=2,sticky='NSEW',padx=10,pady=10)

In [21]:
# Botão para rodar o processo
btn_test = tk.Button(text='Enviar Mensagens',font=('Consolas 10 bold'),command=send_messages)
btn_test.grid(row=5,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
# pensa que enquanto esta rodando a janela, a informação dentro dela esta disponivel

In [22]:
# Label para colocar informações de estado de mensagen atual
lbl_enviando = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
lbl_enviando.grid(row=6,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)

In [23]:
# Scroll text para mostrar a lista
# lbl_enviados = tk.Label(text='',wraplength=500,font=('Consolas 12'),anchor='center')
# lbl_enviados.grid(row=7,column=0,columnspan=3,sticky='NSEW',padx=10,pady=10)
txt_enviados = st.ScrolledText(mainwindow,width = 30, 
                            height = 8, 
                            font = ('Consolas 10'))
txt_enviados.grid(row=7,column = 0, columnspan=3,sticky='NSEW', pady = 10, padx = 10)

txt_enviados.insert(tk.INSERT,'')
# investigar como fazer read only


In [24]:
# janela principal
mainwindow.mainloop()



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [C:\Users\pgetar\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache
C:\Users\pgetar\AppData\Local\Temp/ipykernel_22856/93585002.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  msg_browser = webdriver.Chrome(ChromeDriverManager().install())


1:Pablo: NÃO recebeu a mensagem
2:Pedro with valid message: recebeu a mensagem
3:Pedro Wrong Number: NÃO recebeu a mensagem
4:José valid message: recebeu a mensagem
5:José wrong number: NÃO recebeu a mensagem
6:Maria with message: recebeu a mensagem
7:Maria Wrong number: NÃO recebeu a mensagem
